In [ ]:
"""
EN -> RU translation + summarization
Stable version (no pipeline, CPU, HuggingFace)

Models:
- Translation: facebook/nllb-200-distilled-600M
- Summarization: RussianNLP/FRED-T5-Summarizer

Install:
pip install torch transformers sentencepiece


Text:
Artificial intelligence is rapidly transforming many areas of modern life, including healthcare, education, transportation, and business. One of the key advantages of AI systems is their ability to process large amounts of data much faster than humans, allowing organizations to make more informed decisions.

In healthcare, AI is increasingly used to assist doctors in diagnosing diseases, predicting patient outcomes, and personalizing treatment plans. Machine learning models can analyze medical images, detect patterns in patient records, and identify potential risks at early stages. This can lead to faster diagnoses and improved patient care.

In the business sector, companies use AI to optimize supply chains, automate customer support, and analyze consumer behavior. Chatbots and recommendation systems help businesses reduce costs while improving user experience. However, the adoption of AI also raises concerns about job displacement, data privacy, and ethical decision-making.

Despite these challenges, experts agree that artificial intelligence will continue to play a critical role in the future. Governments and organizations are increasingly focusing on developing regulations and ethical guidelines to ensure that AI technologies are used responsibly and for the benefit of society as a whole.

"""

import textwrap
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    T5ForConditionalGeneration,
    GPT2Tokenizer,
)

# =============================
# НАСТРОЙКИ
# =============================
INPUT_FILE = "input.txt"

DEVICE = "cpu"
MAX_CHARS_TRANSLATE = 900
MAX_CHARS_SUMMARIZE = 1000

# =============================
# ПЕРЕВОД (NLLB)
# =============================
TRANSLATION_MODEL = "facebook/nllb-200-distilled-600M"

print("Загрузка модели перевода...")
trans_tokenizer = AutoTokenizer.from_pretrained(TRANSLATION_MODEL)
trans_model = AutoModelForSeq2SeqLM.from_pretrained(TRANSLATION_MODEL).to(DEVICE)

SRC_LANG = "eng_Latn"
TGT_LANG = "rus_Cyrl"

trans_tokenizer.src_lang = SRC_LANG
FORCED_BOS_ID = trans_tokenizer.convert_tokens_to_ids(TGT_LANG)

def translate_chunk(text: str) -> str:
    inputs = trans_tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        max_length=512
    ).to(DEVICE)

    generated_tokens = trans_model.generate(
        **inputs,
        forced_bos_token_id=FORCED_BOS_ID,
        max_length=512,
        no_repeat_ngram_size=3,
        repetition_penalty=1.1
    )

    return trans_tokenizer.decode(
        generated_tokens[0],
        skip_special_tokens=True
    )

# =============================
# СУММАРИЗАЦИЯ (FRED-T5)
# =============================
SUMM_MODEL = "RussianNLP/FRED-T5-Summarizer"

print("Загрузка модели суммаризации...")
summ_tokenizer = GPT2Tokenizer.from_pretrained(
    SUMM_MODEL,
    eos_token="</s>"
)

summ_model = T5ForConditionalGeneration.from_pretrained(
    SUMM_MODEL
).to(DEVICE)

def summarize_text(text: str, max_len=120, min_len=40) -> str:
    prompt = (
        "Кратко изложи основную мысль текста, "
        "не добавляя новых фактов и источников:\n\n"
        + text
    )

    input_ids = summ_tokenizer.encode(
        "<LM> " + prompt,
        return_tensors="pt",
        truncation=True,
        max_length=512
    ).to(DEVICE)

    output_ids = summ_model.generate(
        input_ids,
        max_length=max_len,
        min_length=min_len,
        num_beams=4,
        no_repeat_ngram_size=3,
        repetition_penalty=1.1
    )

    return summ_tokenizer.decode(
        output_ids[0],
        skip_special_tokens=True
    )


# =============================
# ЧТЕНИЕ ФАЙЛА
# =============================
with open(INPUT_FILE, "r", encoding="utf-8") as f:
    text_en = f.read()

# =============================
# ПЕРЕВОД
# =============================
print("Начинаю перевод...")
chunks_en = textwrap.wrap(text_en, MAX_CHARS_TRANSLATE)

translated_chunks = []
for i, chunk in enumerate(chunks_en, 1):
    print(f"Перевод чанка {i}/{len(chunks_en)}")
    translated_chunks.append(translate_chunk(chunk))

text_ru = " ".join(translated_chunks)

# =============================
# СУММАРИЗАЦИЯ (2 шага)
# =============================
print("Суммаризация (шаг 1)...")
ru_parts = textwrap.wrap(text_ru, MAX_CHARS_SUMMARIZE)
partial_summaries = [
    summarize_text(p, max_len=80, min_len=30)
    for p in ru_parts
]

print("Суммаризация (шаг 2)...")
final_summary = summarize_text(
    " ".join(partial_summaries),
    max_len=150,
    min_len=60
)

# =============================
# РЕЗУЛЬТАТ
# =============================
print("\n" + "=" * 80)
print("=== ПЕРЕВОД ===")
print(text_ru)

print("\n" + "=" * 80)
print("=== ВЫЖИМКА ===")
print(final_summary)
print("=" * 80)


'(ProtocolError('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')), '(Request ID: 5ae1491c-ca9c-4cff-83bd-afcdfe532991)')' thrown while requesting HEAD https://huggingface.co/facebook/nllb-200-distilled-600M/resolve/main/tokenizer_config.json
Retrying in 1s [Retry 1/5].


Загрузка модели перевода...
Загрузка модели суммаризации...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Начинаю перевод...
Перевод чанка 1/2
Перевод чанка 2/2
Суммаризация (шаг 1)...
Суммаризация (шаг 2)...

=== ПЕРЕВОД ===
Искусственный интеллект быстро трансформирует многие области современной жизни, включая здравоохранение, образование, транспорт и бизнес. Одним из ключевых преимуществ систем ИИ является их способность обрабатывать большие объемы данных гораздо быстрее, чем люди, позволяя организациям принимать более информированные решения. В здравоохранении ИИ все чаще используется для оказания помощи врачам в диагностике заболеваний, прогнозировании результатов пациентов и персонализации планов лечения. Модели машинного обучения могут анализировать медицинские изображения, обнаруживать шаблоны в записях пациентов, и выявлять потенциальные риски на ранних стадиях. Это может привести к более быстрой диагноз и улучшению ухода за пациентами. В деловом секторе компании используют ИИ для оптимизации цепочек поставок, автоматизации поддержки клиентов и анализа поведения. Чатботы и системы